In [1]:
import vtk

from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkCommonCore import (
    vtkPoints,
    vtkUnsignedCharArray,
    vtkFloatArray,
    vtkIdList
)
from vtkmodules.vtkFiltersSources import (
    vtkPointSource,
    vtkLineSource,
    vtkPlaneSource,
    vtkDiskSource,
    vtkRegularPolygonSource,
    vtkCubeSource
)
from vtkmodules.vtkCommonDataModel import (
    vtkVertex,
    vtkPolyVertex,
    vtkLine,
    vtkPolyLine,
    vtkTriangle,
    vtkTriangleStrip,
    vtkPixel,
    vtkQuad,
    vtkPolygon,
    vtkCellArray,
    vtkPolyData
)
from vtkmodules.vtkRenderingCore import (
    vtkProperty,
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer
)

In [2]:
def visualize(renderer: vtkRenderer, name = ''):
    # create render window
    renderWindow = vtkRenderWindow()
    renderWindow.SetWindowName(name)
    renderWindow.AddRenderer(renderer)
    
    #create window interaction
    renderWindowInteractor = vtkRenderWindowInteractor()
    renderWindowInteractor.SetRenderWindow(renderWindow)
    
    # Show
    renderWindow.Render()
    renderWindowInteractor.Start()
    
    del renderWindowInteractor, renderWindow

In [3]:
colors = vtkNamedColors()

El conjunto de datos poligonal consta de vértices, polivértices, líneas, polilíneas, polígonos y tiras triangulares. La topología y geometría de los datos poligonales no están estructuradas y las celdas que componen ese conjunto de datos varían en dimensión topológica. El conjunto de datos poligonal forma un puente entre datos, algoritmos y gráficos por computadora de alta velocidad.

Las celdas tridimensionales no pueden incluirse en vtkPolyData, salvo que se haga descomponiéndolas en las anteriores a partir de sus caras.

# Puntos

Las celdas son objetos de clase *vtkVertex* (puntos individuales) o *vtkPolyVertex* (conjuntos de puntos)

## Usando la clase vtkVertex (cada celda es un punto)

In [4]:
# Create the polydata where we will store all the geometric data
pointsPolyData = vtkPolyData()

# Create the geometry of the dataset (the point coordinates)
p0 = [0.0, 0.0, 0.0]
p1 = [1.0, 1.0, 0.0]

points = vtkPoints()
points.InsertNextPoint(p0)
points.InsertNextPoint(p1)

# Add the points to the polydata container
pointsPolyData.SetPoints(points)

In [5]:
# Create the topology of the dataset (one or more cells with vertexes)
# In this example each point forms a cell, and each cell consists of a single point
cells = vtkCellArray()

# Option 1: create explicitly the cells and add them to the CellArray
vertex = vtkVertex()
vertex.GetPointIds().SetId(0, 0)
cells.InsertNextCell(vertex)
vertex = vtkVertex()
vertex.GetPointIds().SetId(0, 1)
cells.InsertNextCell(vertex)

# Option 2: implicitly create the cells by pointing to the points  
#vertices.InsertNextCell(1, [0])
#vertices.InsertNextCell(1, [1])

# Set the vertices we created as the topology of the polydata
pointsPolyData.SetVerts(cells)

In [6]:
print('Number of points: ', pointsPolyData.GetNumberOfPoints())
print('Number of cells: ', pointsPolyData.GetNumberOfCells())

Number of points:  2
Number of cells:  2


In [7]:
# Create attribute data for points and/or cells
# The attribute is stored in a vtk array with so many components as points or cells
# In this example, points and cells have the same attribut data in reverse order

scalars_p = vtkFloatArray()
scalars_p.InsertTuple1(0,0)
scalars_p.InsertTuple1(1,1)

scalars_c = vtkFloatArray()
scalars_c.InsertTuple1(0,1)
scalars_c.InsertTuple1(1,0)

pointsPolyData.GetPointData().SetScalars(scalars_p);
pointsPolyData.GetCellData().SetScalars(scalars_c);

In [8]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(pointsPolyData)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0,1) # Si no se pone, utiliza el rango completo para el color map (rojo a azul)
# Por defecto se usan los datos escalares de los puntos para el color
# mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [9]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Tomato'))
actor.GetProperty().SetPointSize(20)

In [10]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('DarkGreen'))

visualize(renderer, 'Point')

## Usando la clase vtkPolyVertex (cada celda tiene varios puntos, incluso todos)

Usamos la misma geometría que en el ejemplo anterior

In [11]:
# Create the topology of the dataset (one polyvertex)
# In this example each cell (polyvertex) consists of several points
# We might also have several polyvertex, but we'll use just one
cells = vtkCellArray()

polyVertex = vtkPolyVertex()
polyVertex.GetPointIds().SetNumberOfIds(2)
for i in range(0, 2):
    polyVertex.GetPointIds().SetId(i, i)

cells.InsertNextCell(polyVertex);

pointsPolyData.SetVerts(cells)

In [12]:
print('Number of points: ', pointsPolyData.GetNumberOfPoints())
print('Number of cells: ', pointsPolyData.GetNumberOfCells())

Number of points:  2
Number of cells:  1


In [13]:
# Create attribute data for points and/or cells
# The attribute is stored in a vtk array with so many components as points or cells
# In this example, points and cells have the same attribut data in reverse order

scalars_p = vtkFloatArray()
scalars_p.InsertTuple1(0,0)
scalars_p.InsertTuple1(1,1)

scalars_c = vtkFloatArray()
scalars_c.InsertTuple1(0,1)

pointsPolyData.GetPointData().SetScalars(scalars_p);
pointsPolyData.GetCellData().SetScalars(scalars_c);

In [14]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(pointsPolyData)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0,1) # Si no se pone, utiliza el rango completo para el color map (rojo a azul)
# Por defecto se usan los datos escalares de los puntos para el color
mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [15]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Tomato'))
actor.GetProperty().SetPointSize(20)

In [16]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('DarkGreen'))

visualize(renderer, 'Point')

## Objeto fuente de nube de puntos

In [17]:
# Create a point cloud
src = vtkPointSource()
src.SetCenter(0, 0, 0)
src.SetNumberOfPoints(50)
src.SetRadius(5)

In [18]:
# Get PolyData
src.Update()
pointsPolyData = src.GetOutput()

print('Number of points: ', pointsPolyData.GetNumberOfPoints())
print('Number of cells: ', pointsPolyData.GetNumberOfCells())

Number of points:  50
Number of cells:  1


In [19]:
scalars_p = vtkFloatArray()

for p in range(0,50):
    scalars_p.InsertTuple1(p,p)
    
pointsPolyData.GetPointData().SetScalars(scalars_p);

In [20]:
# mapper
mapper = vtkPolyDataMapper()
mapper.SetInputData(pointsPolyData)
#mapper.SetInputConnection(src.GetOutputPort()) # si usamos directamente la fuente

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0,49)

In [21]:
# actor
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Tomato'))
actor.GetProperty().SetPointSize(4)

In [22]:
renderer = vtkRenderer()

renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('DarkGreen'))

visualize(renderer, 'PointSource')

# Líneas

Las celdas son objetos de clase *vtkLine* (líneass individuales) o *vtkPolyLine* (conjuntos de líneas)

## Usando la clase vtkLine (cada celda es una línea)

In [24]:
# Create a polydata to store everything in
linesPolyData = vtkPolyData()

# Create five points.
origin = [0.0, 0.0, 0.0]
p0 = [1.0, 0.0, 0.0]
p1 = [0.0, 1.0, 0.0]
p2 = [0.0, 1.0, 2.0]
p3 = [1.0, 2.0, 3.0]

# Create a vtkPoints object and store the points in it
points = vtkPoints()
points.InsertNextPoint(origin)
points.InsertNextPoint(p0)
points.InsertNextPoint(p1)
points.InsertNextPoint(p2)
points.InsertNextPoint(p3)

# Add the points to the dataset
linesPolyData.SetPoints(points)

In [25]:
# Create a cell array to store the lines in and add the lines to it
lines = vtkCellArray()

for i in range(0, 4):
    line = vtkLine()
    line.GetPointIds().SetId(0, i)
    line.GetPointIds().SetId(1, i + 1)
    lines.InsertNextCell(line)

# Add the lines to the dataset
linesPolyData.SetLines(lines)

In [26]:
scalars_p = vtkFloatArray()

for p in range(0,5):
    scalars_p.InsertTuple1(p,p)
    
linesPolyData.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

for c in range(0,4):
    scalars_c.InsertTuple1(c,c)
    
linesPolyData.GetCellData().SetScalars(scalars_c);

In [30]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(linesPolyData)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 4)

# Visualizar celdas
#mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color
#mapper.SetScalarRange(0,3)

In [31]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetLineWidth(4)
actor.GetProperty().SetColor(colors.GetColor3d('Peacock'))

In [32]:
renderer = vtkRenderer()
renderer.AddActor(actor)

renderer.ResetCamera()
renderer.GetActiveCamera().Azimuth(30)
renderer.GetActiveCamera().Elevation(30)
renderer.ResetCameraClippingRange()

renderer.SetBackground(colors.GetColor3d('Silver'))

visualize(renderer, 'LongLine')

### Otro ejemplo con varias celdas vtkLine

Definimos colores, no escalares que se visualizan con mapas de color, como en los ejemplos anteriores

In [33]:
# Create the polydata where we will store all the geometric data
linesPolyData = vtkPolyData()

# Create three points
origin = [0.0, 0.0, 0.0]
p0 = [1.0, 0.0, 0.0]
p1 = [0.0, 1.0, 0.0]

# Create a vtkPoints container and store the points in it
pts = vtkPoints()
pts.InsertNextPoint(origin)
pts.InsertNextPoint(p0)
pts.InsertNextPoint(p1)

# Add the points to the polydata container
linesPolyData.SetPoints(pts)

In [34]:
# Create the first line (between Origin and P0)
line0 = vtkLine()
line0.GetPointIds().SetId(0, 0)  # the second 0 is the index of the Origin in linesPolyData's points
line0.GetPointIds().SetId(1, 1)  # the second 1 is the index of P0 in linesPolyData's points

# Create the second line (between Origin and P1)
line1 = vtkLine()
line1.GetPointIds().SetId(0, 0)  # the second 0 is the index of the Origin in linesPolyData's points
line1.GetPointIds().SetId(1, 2)  # 2 is the index of P1 in linesPolyData's points

# Create a vtkCellArray container and store the lines in it
lines = vtkCellArray()
lines.InsertNextCell(line0)
lines.InsertNextCell(line1)

# Add the lines to the polydata container
linesPolyData.SetLines(lines)

In [35]:
# Create a vtkUnsignedCharArray container and store the colors in it
colorTuples = vtkUnsignedCharArray()
colorTuples.SetNumberOfComponents(3)
try:
    colorTuples.InsertNextTupleValue(colors.GetColor3ub("Tomato"))
    colorTuples.InsertNextTupleValue(colors.GetColor3ub("Mint"))
except AttributeError:
    # For compatibility with new VTK generic data arrays.
    colorTuples.InsertNextTypedTuple(colors.GetColor3ub("Tomato"))
    colorTuples.InsertNextTypedTuple(colors.GetColor3ub("Mint"))

# Color the lines.
# SetScalars() automatically associates the values in the data array passed as parameter
# to the elements in the same indices of the cell data array on which it is called.
# This means the first component (red) of the colors array
# is matched with the first component of the cell array (line 0)
# and the second component (green) of the colors array
# is matched with the second component of the cell array (line 1)
    
linesPolyData.GetCellData().SetScalars(colorTuples);

In [36]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(linesPolyData)

In [37]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetLineWidth(4)

In [38]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d("SlateGray"))

visualize(renderer, 'ColoredLines')

## PolyLine (cada celda tiene varias líneas, incluso todas)

In [39]:
# Create a polydata to store everything in
polyData = vtkPolyData()

# Create five points.
origin = [0.0, 0.0, 0.0]
p0 = [1.0, 0.0, 0.0]
p1 = [0.0, 1.0, 0.0]
p2 = [0.0, 1.0, 2.0]
p3 = [1.0, 2.0, 3.0]

# Create a vtkPoints object and store the points in it
points = vtkPoints()
points.InsertNextPoint(origin)
points.InsertNextPoint(p0)
points.InsertNextPoint(p1)
points.InsertNextPoint(p2)
points.InsertNextPoint(p3)

# Add the points to the dataset
polyData.SetPoints(points)

In [45]:
# Create a cell array to store the lines in and add the lines to it
cells = vtkCellArray()

polyLine = vtkPolyLine()
polyLine.GetPointIds().SetNumberOfIds(5)
for i in range(0, 5):
    polyLine.GetPointIds().SetId(i, i)

cells.InsertNextCell(polyLine)

# Add the lines to the dataset
polyData.SetLines(cells)

In [47]:
print('Number of points: ', polyData.GetNumberOfPoints())
print('Number of cells: ', polyData.GetNumberOfCells())

Number of points:  5
Number of cells:  1


In [53]:
scalars_p = vtkFloatArray()

for p in range(0,5):
    scalars_p.InsertTuple1(p,p)
    
polyData.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

scalars_c.InsertTuple1(0,0)
    
polyData.GetCellData().SetScalars(scalars_c);

In [54]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(polyData)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 4)

# Visualizar celdas
mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [55]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Tomato'))

In [56]:
# Setup render window, renderer, and interactor
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('DarkOliveGreen'))

visualize(renderer, 'PolyLine')

## Objeto fuente de línea

In [57]:
lineSource = vtkLineSource()

# Create two points, P0 and P1
p0 = [1.0, 0.0, 0.0]
p1 = [0.0, 1.0, 0.0]

lineSource.SetPoint1(p0)
lineSource.SetPoint2(p1)

In [58]:
# Get PolyData
lineSource.Update()
linePolyData = lineSource.GetOutput()

print('Number of points: ', linePolyData.GetNumberOfPoints())
print('Number of cells: ', linePolyData.GetNumberOfCells())

Number of points:  2
Number of cells:  1


In [67]:
scalars_p = vtkFloatArray()

scalars_p.InsertTuple1(0,0)
scalars_p.InsertTuple1(1,1)

linePolyData.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()
scalars_c.InsertTuple1(0,0)
    
linePolyData.GetCellData().SetScalars(scalars_c);

In [68]:
mapper = vtkPolyDataMapper()
mapper.SetInputConnection(lineSource.GetOutputPort())

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 1)

# Visualizar celdas
#mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [69]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetLineWidth(4)
actor.GetProperty().SetColor(colors.GetColor3d("Peacock"))

In [71]:
renderer = vtkRenderer()

renderer.SetBackground(colors.GetColor3d("Silver"))
renderer.AddActor(actor)

visualize(renderer, 'Line')

# Triángulo

In [72]:
# Create a polydata object
trianglePolyData = vtkPolyData()

# Create a triangle
points = vtkPoints()
points.InsertNextPoint(1.0, 0.0, 0.0)
points.InsertNextPoint(0.0, 0.0, 0.0)
points.InsertNextPoint(0.0, 1.0, 0.0)

trianglePolyData.SetPoints(points)

In [73]:
cells = vtkCellArray()

triangle = vtkTriangle()
triangle.GetPointIds().SetId(0, 0)
triangle.GetPointIds().SetId(1, 1)
triangle.GetPointIds().SetId(2, 2)

cells.InsertNextCell(triangle)

trianglePolyData.SetPolys(cells)

In [85]:
scalars_p = vtkFloatArray()

for p in range(0,3):
    scalars_p.InsertTuple1(p,p)
    
trianglePolyData.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

scalars_c.InsertTuple1(0,1)
    
trianglePolyData.GetCellData().SetScalars(scalars_c);

In [89]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(trianglePolyData)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 2)

# Visualizar celdas
#mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [90]:
actor = vtkActor()
actor.GetProperty().SetColor(colors.GetColor3d('PeachPuff'))
actor.SetMapper(mapper)

In [91]:
# Create a renderer, render window, and an interactor
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('DarkGreen'))

visualize(renderer, 'Triangle')

# Tira de triángulos (Triangle strip)

In [20]:
polydata = vtkPolyData()

points = vtkPoints()
points.InsertNextPoint(0, 0, 0.5)
points.InsertNextPoint(0, 1, 0)
points.InsertNextPoint(1, 0, 0)
points.InsertNextPoint(1.5, 1, 0)
points.InsertNextPoint(1.5, 1, 1)

polydata.SetPoints(points)

In [21]:
cells = vtkCellArray()

triangleStrip = vtkTriangleStrip()
triangleStrip.GetPointIds().SetNumberOfIds(5)
triangleStrip.GetPointIds().SetId(0, 0)
triangleStrip.GetPointIds().SetId(1, 1)
triangleStrip.GetPointIds().SetId(2, 2)
triangleStrip.GetPointIds().SetId(3, 3)
triangleStrip.GetPointIds().SetId(4, 4)

cells.InsertNextCell(triangleStrip)

polydata.SetStrips(cells)

In [22]:
scalars_p = vtkFloatArray()

for p in range(0,5):
    scalars_p.InsertTuple1(p,p)
    
polydata.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

scalars_c.InsertTuple1(0,1)
    
polydata.GetCellData().SetScalars(scalars_c);

In [23]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(polydata)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 4)

# Visualizar celdas
#mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [24]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('PeachPuff'))
#actor.GetProperty().SetRepresentationToWireframe()

In [25]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('DarkGreen'))

visualize(renderer, 'TriangleStrip')

# Píxel

In [26]:
polydata = vtkPolyData()

# Create four points (must be in counter clockwise order)
p0 = [0.0, 0.0, 0.0]
p1 = [1.0, 0.0, 0.0]
p2 = [1.0, 1.0, 0.0]
p3 = [0.0, 1.0, 0.0]

# Add the points to a vtkPoints object
points = vtkPoints()
points.InsertNextPoint(p0)
points.InsertNextPoint(p1)
points.InsertNextPoint(p2)
points.InsertNextPoint(p3)

polydata.SetPoints(points)

In [35]:
# Create a cell array to store the quad in
cells = vtkCellArray()

# Create a quad on the four points
pixel = vtkPixel()
pixel.GetPointIds().SetId(0, 0)
pixel.GetPointIds().SetId(1, 1)
pixel.GetPointIds().SetId(2, 2)
pixel.GetPointIds().SetId(3, 3)

cells.InsertNextCell(pixel)

polydata.SetPolys(cells)

In [38]:
scalars_p = vtkFloatArray()

for p in range(0,3):
    scalars_p.InsertTuple1(p,p)
    
polydata.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

scalars_c.InsertTuple1(0,1)
    
polydata.GetCellData().SetScalars(scalars_c);

In [49]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(polydata)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 2)

# Visualizar celdas
#mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [50]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Silver'))

In [51]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('Gray'))

visualize(renderer, 'Quad')

# Cuadrilátero

In [52]:
polydata = vtkPolyData()

# Create four points (must be in counter clockwise order)
p0 = [0.0, 0.0, 0.0]
p1 = [2.0, 0.0, 0.0]
p2 = [1.0, 1.0, 0.0]
p3 = [0.5, 1.0, 0.0]

# Add the points to a vtkPoints object
points = vtkPoints()
points.InsertNextPoint(p0)
points.InsertNextPoint(p1)
points.InsertNextPoint(p2)
points.InsertNextPoint(p3)

polydata.SetPoints(points)

In [53]:
# Create a cell array to store the quad in
cells = vtkCellArray()

# Create a quad on the four points
quad = vtkQuad()
quad.GetPointIds().SetId(0, 0)
quad.GetPointIds().SetId(1, 1)
quad.GetPointIds().SetId(2, 2)
quad.GetPointIds().SetId(3, 3)

cells.InsertNextCell(quad)

polydata.SetPolys(cells)

In [57]:
scalars_p = vtkFloatArray()

for p in range(0,3):
    scalars_p.InsertTuple1(p,p)
    
polydata.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

scalars_c.InsertTuple1(0,1)
    
polydata.GetCellData().SetScalars(scalars_c);

In [58]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(polydata)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 2)

# Visualizar celdas
#mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [59]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Silver'))

In [61]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('Gray'))

visualize(renderer, 'Quad')

## Objeto fuente plano

In [62]:
# Create a plane
planeSource = vtkPlaneSource()
planeSource.SetCenter(1.0, 0.0, 0.0)
planeSource.SetNormal(1.0, 0.0, 1.0)
planeSource.Update()

# Get PolyData
plane = planeSource.GetOutput()

In [63]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(plane)

In [64]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Banana'))

In [65]:
# Create a renderer, render window and interactor
renderer = vtkRenderer()

renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('BkgColor'))
    
renderer.AddActor(actor)
colors.SetColor('BkgColor', [26, 51, 77, 255])
renderer.SetBackground(colors.GetColor3d('BkgColor'))

visualize(renderer, 'Plane')

# Polígono

In [66]:
# Create a PolyData
polygonPolyData = vtkPolyData()

# Setup four points
points = vtkPoints()
points.InsertNextPoint(0.0, 0.0, 0.0)
points.InsertNextPoint(1.0, 0.0, 0.0)
points.InsertNextPoint(1.0, 1.0, 0.0)
points.InsertNextPoint(0.0, 1.0, 0.0)

polygonPolyData.SetPoints(points)

In [67]:
# Add the polygon to a list of polygons
polygons = vtkCellArray()
    
# Create the polygon
polygon = vtkPolygon()
polygon.GetPointIds().SetNumberOfIds(4)  # make a quad
polygon.GetPointIds().SetId(0, 0)
polygon.GetPointIds().SetId(1, 1)
polygon.GetPointIds().SetId(2, 2)
polygon.GetPointIds().SetId(3, 3)

polygons.InsertNextCell(polygon)

polygonPolyData.SetPolys(polygons)

In [68]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(polygonPolyData)

In [69]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(colors.GetColor3d('Silver'))

In [70]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('Salmon'))

visualize(renderer, 'Polygon')

## Objeto fuente disco

In [82]:
diskSource = vtkDiskSource()

In [83]:
# Get PolyData
diskSource.Update()
diskPolyData = diskSource.GetOutput()

print('Number of points: ', diskPolyData.GetNumberOfPoints())
print('Number of cells: ', diskPolyData.GetNumberOfCells())

Number of points:  12
Number of cells:  6


In [101]:
scalars_p = vtkFloatArray()

for p in range(0,11):
    scalars_p.InsertTuple1(p,p)
    
diskPolyData.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

for c in range(0,6):
    scalars_c.InsertTuple1(c,c)
    
diskPolyData.GetCellData().SetScalars(scalars_c);

In [102]:
mapper = vtkPolyDataMapper()
mapper.SetInputConnection(diskSource.GetOutputPort())

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 11)

# Visualizar celdas
mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color
mapper.SetScalarRange(0, 5)

In [103]:
actor = vtkActor()
actor.GetProperty().SetColor(colors.GetColor3d("Cornsilk"))
actor.SetMapper(mapper)

In [104]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d("Gray"))

visualize(renderer, 'Disk')

## Objeto fuente polígono regular

In [105]:
# Create a pentagon
polygonSource = vtkRegularPolygonSource()

polygonSource.SetNumberOfSides(5)
polygonSource.SetRadius(5)
polygonSource.SetCenter(0, 0, 0)

In [106]:
mapper = vtkPolyDataMapper()
mapper.SetInputConnection(polygonSource.GetOutputPort())

In [107]:
actor = vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().EdgeVisibilityOn()
actor.GetProperty().SetLineWidth(5)
actor.GetProperty().SetColor(colors.GetColor3d('Banana'))

#cara trasera con color distinto a la delantera
back = vtkProperty()
back.SetColor(colors.GetColor3d('Tomato'))
actor.SetBackfaceProperty(back)

In [108]:
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d('Silver'))

visualize(renderer, 'RegularPolygonSource')

# Cubo

## Construcción programática del cubo

In [109]:
hexaPolyData = vtkPolyData()

# Set up the eight points
# For the hexahedron setup the coordinates of eight points.
# The two faces must be in counter clockwise order as viewed from the
# outside.
points = vtkPoints()

pointCoordinates = list()
pointCoordinates.append([0.0, 0.0, 0.0])  # Face 1
pointCoordinates.append([1.0, 0.0, 0.0])
pointCoordinates.append([1.0, 1.0, 0.0])
pointCoordinates.append([0.0, 1.0, 0.0])
pointCoordinates.append([0.0, 0.0, 1.0])  # Face 2
pointCoordinates.append([1.0, 0.0, 1.0])
pointCoordinates.append([1.0, 1.0, 1.0])
pointCoordinates.append([0.0, 1.0, 1.0])


for i in range(0, len(pointCoordinates)):
    points.InsertNextPoint(pointCoordinates[i])

hexaPolyData.SetPoints(points)

In [110]:
#Nota: si quisiéramos utilizar una celda 3D (ej vtkHexahedron) tendríamos que utilizar "unstructured grids", no vtkPolyD

def mkVtkIdList(it):
    """
    Makes a vtkIdList from a Python iterable. 

    """
    vil = vtkIdList()
    for i in it:
        vil.InsertNextId(int(i))
    return vil

# pts = array of 6 4-tuples of vtkIdType (int) representing the faces
#     of the cube in terms of the above vertices
pts = [(0, 3, 2, 1), (4, 5, 6, 7), (0, 1, 5, 4),
        (1, 2, 6, 5), (2, 3, 7, 6), (3, 0, 4, 7)]

# Creat cell array with six components, one for each face
cells = vtkCellArray()

for pt in pts:
    cells.InsertNextCell(mkVtkIdList(pt))

hexaPolyData.SetPolys(cells)

In [111]:
scalars = vtkFloatArray()

# Set point data
for i, _ in enumerate(pointCoordinates):
    scalars.InsertTuple1(i, i)
    
hexaPolyData.GetPointData().SetScalars(scalars)

# Set cell data
for i, _ in enumerate(pts):
    scalars.InsertTuple1(i, i)
    
hexaPolyData.GetCellData().SetScalars(scalars)

hexaPolyData.GetPointData().SetScalars(scalars);
hexaPolyData.GetCellData().SetScalars(scalars);

In [112]:
mapper = vtkPolyDataMapper()
mapper.SetInputData(hexaPolyData)

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0,5) # Si no se pone, utiliza el rango completo para el color map (rojo a azul)
# Por defecto se usan los datos escalares de los puntos para el color
#mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color

In [113]:
actor = vtkActor()
actor.GetProperty().SetColor(colors.GetColor3d("PeachPuff"))
actor.SetMapper(mapper)

In [114]:
# Set the background color.
colors.SetColor("BkgColor", [51, 77, 102, 255])
    
renderer = vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(colors.GetColor3d("BkgColor"))
renderer.ResetCamera()
renderer.GetActiveCamera().Azimuth(30)
renderer.GetActiveCamera().Elevation(30)

visualize(renderer, 'Hexahedron')

## Objeto fuente cubo

Los objetos geométricos simples creados por filtros de fuente, tienen tantas celdas como caras, que son poligonales.

Pensar cómo hacer una cara transparente, y poner colores diferentes en el interior y exterior

In [4]:
# Create cube.
cube = vtkCubeSource()

In [5]:
# Get PolyData
cube.Update()
cubePolyData = cube.GetOutput()

print('Number of points: ', cubePolyData.GetNumberOfPoints())
print('Number of cells: ', cubePolyData.GetNumberOfCells())

Number of points:  24
Number of cells:  6


In [11]:
scalars_p = vtkFloatArray()

for p in range(0,24):
    scalars_p.InsertTuple1(p,p)
    
cubePolyData.GetPointData().SetScalars(scalars_p);


scalars_c = vtkFloatArray()

for c in range(0,6):
    scalars_c.InsertTuple1(c,c)
    
cubePolyData.GetCellData().SetScalars(scalars_c);

In [12]:
# mapper
mapper = vtkPolyDataMapper()
mapper.SetInputData(cube.GetOutput())

mapper.SetScalarVisibility(1) # controla si se usa el color del actor o el de los datos escalares
mapper.SetScalarRange(0, 23)

# Visualizar celdas
mapper.SetScalarModeToUseCellData() # usar datos escalares de las celdas para el color
mapper.SetScalarRange(0, 5)

In [13]:
# Actor.
cubeActor = vtkActor()
cubeActor.SetMapper(mapper)
cubeActor.GetProperty().SetColor(colors.GetColor3d('Banana'))

In [14]:
renderer = vtkRenderer()

renderer.AddActor(cubeActor)
renderer.ResetCamera()
renderer.GetActiveCamera().Azimuth(30)
renderer.GetActiveCamera().Elevation(30)
renderer.ResetCameraClippingRange()
renderer.SetBackground(colors.GetColor3d('Silver'))

visualize(renderer, 'Cube')